## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras import regularizers
from keras.layers import Activation
from keras.callbacks import LearningRateScheduler


from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)
print(y_train.shape) #(50000, 1)
print(x_test.shape)  #(10000, 32, 32, 3)
print(y_test.shape)  #(10000, 1)

## Normalize Data
def normalize(X_train, X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test, mean, std
    
    
## Normalize Training and Testset    
x_train, x_test, mean_train, std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray() #OneHotEncoder會轉出scipy.csr_matrix資料結構用.toarray()轉numpy array
print(y_train.shape)
y_test=one_hot.transform(y_test).toarray()
print(y_test.shape)

(50000, 10)
(10000, 10)


In [4]:
def schduler(epoch, lr):
    init_lr= 1e-3

    if epoch < 12:
        return init_lr
    elif epoch >=12 and epoch < 15:
        return init_lr /2
    elif epoch >= 15 and epoch <20:
        return init_lr / 10

In [16]:

l2 = 5e-4
callbacks = [
    LearningRateScheduler(schduler, verbose=1)
]

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3,3), 
                             input_shape = (32, 32, 3), activation='relu', kernel_regularizer=regularizers.l2(l2)))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001))

#'''自己決定MaxPooling2D放在哪裡'''

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3,3), activation='relu', kernel_regularizer=regularizers.l2(l2)))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001))

classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合 
classifier.add(Convolution2D(64, kernel_size=(3,3), activation='relu', kernel_regularizer=regularizers.l2(l2)))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001))

#卷積組合 
classifier.add(Convolution2D(64, kernel_size=(3,3), activation='relu', kernel_regularizer=regularizers.l2(l2)))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001))


#卷積組合 
classifier.add(Convolution2D(128, kernel_size=(3,3), activation='relu', kernel_regularizer=regularizers.l2(l2)))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001))

#卷積組合 
classifier.add(Convolution2D(128, kernel_size=(3,3), activation='relu', kernel_regularizer=regularizers.l2(l2)))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(l2)))
classifier.add(Dropout(0.5))
#輸出
classifier.add(Dense(output_dim=10,activation='softmax', kernel_regularizer=regularizers.l2(l2)))
classifier.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_25 (Batc (None, 30, 30, 32)        128       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
batch_normalization_26 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 12, 12, 64)        18496     
_________________________________________________________________
batch_normalization_27 (Batc (None, 12, 12, 64)       

In [17]:
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=128,epochs=20, validation_data=(x_test, y_test), callbacks=callbacks)

Train on 50000 samples, validate on 10000 samples
Epoch 1/20

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
50000/50000 [==============================] - 8s 159us/step - loss: 2.1107 - accuracy: 0.4119 - val_loss: 1.8182 - val_accuracy: 0.4824
Epoch 2/20

Epoch 00002: LearningRateScheduler setting learning rate to 0.001.
50000/50000 [==============================] - 7s 138us/step - loss: 1.5180 - accuracy: 0.5792 - val_loss: 1.3465 - val_accuracy: 0.6381
Epoch 3/20

Epoch 00003: LearningRateScheduler setting learning rate to 0.001.
50000/50000 [==============================] - 7s 139us/step - loss: 1.2960 - accuracy: 0.6568 - val_loss: 1.2261 - val_accuracy: 0.6810
Epoch 4/20

Epoch 00004: LearningRateScheduler setting learning rate to 0.001.
50000/50000 [==============================] - 7s 137us/step - loss: 1.1551 - accuracy: 0.7040 - val_loss: 1.1394 - val_accuracy: 0.7056
Epoch 5/20

Epoch 00005: LearningRateScheduler setting learning rate to 0.001.
50000/5

In [ ]:
# 4 conv
# Epoch 20/20
# 50000/50000 [==============================] - 5s 107us/step - loss: 0.2093 - accuracy: 0.9250 - val_loss: 1.0676 - val_accuracy: 0.7571

# 6 conv
# Epoch 20/20
# 50000/50000 [==============================] - 6s 128us/step - loss: 0.1691 - accuracy: 0.9415 - val_loss: 0.8620 - val_accuracy: 0.7954

# 6 conv with regularizers
# Epoch 20/20
# 50000/50000 [==============================] - 7s 131us/step - loss: 1.0214 - accuracy: 0.8168 - val_loss: 1.1655 - val_accuracy: 0.7739

# 6 conv with ref and 


## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [18]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.5207659e-04, 2.9897876e-06, 2.3086229e-03, 9.4982153e-01,
        3.9112419e-02, 5.4175896e-04, 7.2014830e-03, 3.2285438e-06,
        8.4389967e-04, 1.2011511e-05]], dtype=float32)

In [19]:
test_img = x_test[0,...]
test_img = test_img.reshape(1, test_img.shape[0], test_img.shape[1], test_img.shape[2])
test_label = y_test[0]

print(test_label)
print(classifier.predict(test_img))


[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[[1.3463177e-06 5.5094972e-07 5.6494018e-06 9.9960023e-01 6.2131244e-07
  1.1880239e-04 2.5179121e-04 2.1903136e-06 1.0858335e-05 7.9874062e-06]]
